# Setup



In [1]:
import tensorflow as tf

import numpy as np
import os
import time

# Read data

In [2]:
path_to_file = "./aud_data.txt"

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 978138 characters


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

66 unique characters


In [5]:
text[0:200]

'လူတိုင်း မှာ မ ရှိ သင့် တဲ့ အရာ က မနာလိုစိတ် ပဲ ဖြစ် တယ်\nမနာလိုစိတ် ဆိုတာ ဘယ်သူ့ ကို မှ ကောင်းကျိုး မ ပေး ဘူး\nတခြား သူ ကို မနာလို တဲ့ အခါ သူ ဟာ အသုံးမကျ တဲ့ လူ ဖြစ် သွား တတ် တယ်\nကိုယ့် ပြိုင်ဘက် အတွက်'

# Process the text

## Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The tf.keras.layers.StringLookup layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [6]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

# Create training examples and targets

In [7]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [8]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

လ
ူ
တ
ိ
ု
င
်
း
 
မ


In [9]:
seq_length = 200

The batch method lets you easily convert these individual characters to sequences of the desired size.

In [10]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. At each time step the input is the current character and the label is the next character.

In [11]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [12]:
dataset = sequences.map(split_input_target)

# Create training batches

Use tf.data to split the text into manageable sequences

But before feeding this data into the model, it is needed to shuffle the data and pack it into batches.




In [13]:
# Batch size
BATCH_SIZE =  64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 200), dtype=tf.int64, name=None), TensorSpec(shape=(64, 200), dtype=tf.int64, name=None))>

# Build The Model

This model has three layers:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;

tf.keras.layers.GRU: A type of RNN with size units=rnn_units

tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [14]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [16]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 200, 67) # (batch_size, sequence_length, vocab_size)


In [18]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  17152     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  68675     
                                                                 
Total params: 4024131 (15.35 MB)
Trainable params: 4024131 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Train the model

In [19]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [20]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
EPOCHS = 100

In [21]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
76/76 [==============================] - 14s 107ms/step - loss: 2.9680
Epoch 2/100
76/76 [==============================] - 11s 114ms/step - loss: 1.8786
Epoch 3/100
76/76 [==============================] - 10s 106ms/step - loss: 1.6708
Epoch 4/100
76/76 [==============================] - 10s 108ms/step - loss: 1.5170
Epoch 5/100
76/76 [==============================] - 10s 109ms/step - loss: 1.3984
Epoch 6/100
76/76 [==============================] - 11s 110ms/step - loss: 1.3058
Epoch 7/100
76/76 [==============================] - 10s 111ms/step - loss: 1.2338
Epoch 8/100
76/76 [==============================] - 11s 115ms/step - loss: 1.1787
Epoch 9/100
76/76 [==============================] - 10s 116ms/step - loss: 1.1344
Epoch 10/100
76/76 [==============================] - 10s 118ms/step - loss: 1.0960
Epoch 11/100
76/76 [==============================] - 10s 118ms/step - loss: 1.0637
Epoch 12/100
76/76 [==============================] - 10s 113ms/step - loss: 1.0348
E

# Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

In [26]:
model = MyModel(vocab_size, embedding_dim, rnn_units)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [27]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [28]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars,1.0)

In [29]:
import time
import tensorflow as tf


def generate_text_until_sentences(model, initial_text, num_sentences=1, max_length=100):
    start = time.time()
    states = None
    next_char = tf.constant([initial_text])
    result = [next_char]

    sentence_count = 0

    while sentence_count < num_sentences:
        next_char, states = model.generate_one_step(next_char, states=states)
        result.append(next_char)

        # Check if the generated character is the end of a sentence
        generated_text = next_char[0].numpy().decode('utf-8')
        if generated_text and generated_text[-1] in ['\n']:
            sentence_count += 1

        # Limit the length of the generated text
        if len(result) >= max_length:
            break

    result = tf.strings.join(result)
    end = time.time()
    print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
    print('\nRun time:', end - start)

generate_text_until_sentences(one_step_model, 'နေကောင်း ကြ ', num_sentences=1, max_length=200)

နေကောင်း ကြ တာ ပဲ
 

________________________________________________________________________________

Run time: 0.9716076850891113


In [30]:
generate_text_until_sentences(one_step_model, 'အဆင်ပြေ ကြ တယ် ', num_sentences=1, max_length=200)

အဆင်ပြေ ကြ တယ် ပေါ့
 

________________________________________________________________________________

Run time: 0.021728515625


In [31]:
generate_text_until_sentences(one_step_model, 'နံပါတ် တစ် ', num_sentences=1, max_length=200)

နံပါတ် တစ် က ကို ကောင်း တယ်
 

________________________________________________________________________________

Run time: 0.07864689826965332


In [32]:
generate_text_until_sentences(one_step_model, 'သာမာန် လူ တွေ က မျက်လုံး နှစ်လုံး ဖွင့် ပြီး ', num_sentences=1, max_length=200)

သာမာန် လူ တွေ က မျက်လုံး နှစ်လုံး ဖွင့် ပြီး တော့ သူ့ ကို ဂရုစိုက် ခြင်း တွေ ရှိ တယ်
 

________________________________________________________________________________

Run time: 0.12487626075744629


In [33]:
generate_text_until_sentences(one_step_model, 'အောင်မြင်ခြင်း နဲ့ ကျရှုံးခြင်း တွေ ရဲ့ ကြား မှာ မ ဖြစ် နိုင် တဲ့ စည်း တွေ ', num_sentences=1, max_length=200)

အောင်မြင်ခြင်း နဲ့ ကျရှုံးခြင်း တွေ ရဲ့ ကြား မှာ မ ဖြစ် နိုင် တဲ့ စည်း တွေ ဖယ် ပေး သော အသက်အရွယ် တွေ ကို တာဝန် သိ စွာ တွဲ နေ ခဲ့ မှာ ပဲ
 

________________________________________________________________________________

Run time: 0.1983342170715332


In [34]:
generate_text_until_sentences(one_step_model, 'မင်း ဘာ ကြောင့် အောင်မြင် ', num_sentences=1, max_length=200)

မင်း ဘာ ကြောင့် အောင်မြင် မှု ရ သည် အကြောင်း ကျွမ်းကျပ် အရှုံး တွေ ကို ဝါသနာ တွေ အများကြီး မ ကြိုးစား ခဲ့ စေ ချင် ဘူး
 

________________________________________________________________________________

Run time: 0.29592370986938477


In [35]:
# update temperature
one_step_model_upd = OneStep(model, chars_from_ids, ids_from_chars,0.5)

In [36]:
generate_text_until_sentences(one_step_model_upd, 'နေကောင်း ကြ ', num_sentences=1, max_length=200)

နေကောင်း ကြ တော့ တယ်
 

________________________________________________________________________________

Run time: 0.9389612674713135


In [37]:
generate_text_until_sentences(one_step_model_upd, 'အဆင်ပြေ ကြ တယ် ', num_sentences=1, max_length=200)

အဆင်ပြေ ကြ တယ် ပေါ့
 

________________________________________________________________________________

Run time: 0.022274255752563477


In [38]:
generate_text_until_sentences(one_step_model_upd, 'နံပါတ် တစ် ', num_sentences=1, max_length=200)

နံပါတ် တစ် က ကြာ တယ် လေ
 

________________________________________________________________________________

Run time: 0.04493832588195801


In [39]:
generate_text_until_sentences(one_step_model_upd, 'သာမာန် လူ တွေ က မျက်လုံး နှစ်လုံး ဖွင့် ပြီး ', num_sentences=1, max_length=200)

သာမာန် လူ တွေ က မျက်လုံး နှစ်လုံး ဖွင့် ပြီး လောကကြီး ကို ကြည့် တယ်
 

________________________________________________________________________________

Run time: 0.1061711311340332


In [40]:
generate_text_until_sentences(one_step_model_upd, 'အောင်မြင်ခြင်း နဲ့ ကျရှုံးခြင်း တွေ ရဲ့ ကြား မှာ မ ဖြစ် နိုင် တဲ့ စည်း တွေ ', num_sentences=1, max_length=200)

အောင်မြင်ခြင်း နဲ့ ကျရှုံးခြင်း တွေ ရဲ့ ကြား မှာ မ ဖြစ် နိုင် တဲ့ စည်း တွေ ခြား မ ထား ဘူး
 

________________________________________________________________________________

Run time: 0.0512235164642334


In [41]:
generate_text_until_sentences(one_step_model_upd, 'မင်း ဘာ ကြောင့် အောင်မြင် ', num_sentences=1, max_length=200)

မင်း ဘာ ကြောင့် အောင်မြင် ဖို့ အတွက် အခွင့်အရေး ရှိ လာ မှာ ပါ
 

________________________________________________________________________________

Run time: 0.10771512985229492


# Export the Generation

In [42]:
tf.saved_model.save(one_step_model_upd, 'one_step')

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
mv training_checkpoints /content/drive/MyDrive/

In [45]:
!mv one_step /content/drive/MyDrive/